In [1]:
import pandas as pd

In [2]:
#create the sec codes dataset
sic_codes = pd.read_csv('corpwatch_api_tables_csv/sic_codes.csv', sep = "\t", escapechar='\\')
sic_sectors = pd.read_csv('corpwatch_api_tables_csv/sic_sectors.csv', sep = "\t", escapechar='\\')

In [3]:
sic_sectors_codes = pd.merge(sic_codes,
                     sic_sectors,
                     on='sic_sector', 
                     how='left')

In [4]:
sic_sectors_codes.to_csv("sector_output.csv")

## Read in the companies and merge with the codes

In [5]:
company_info = pd.read_csv('corpwatch_api_tables_csv/company_info.csv', sep = "\t", escapechar='\\',  low_memory=False)

In [6]:
company_info.columns

Index(['row_id', 'cw_id', 'most_recent', 'year', 'cik', 'irs_number',
       'best_location_id', 'sic_code', 'industry_name', 'sic_sector',
       'sector_name', 'source_type', 'source_id', 'num_parents',
       'num_children', 'top_parent_id', 'company_name', 'max_year', 'min_year',
       'no_sic'],
      dtype='object')

In [7]:
company_info.head()

,row_id,cw_id,most_recent,year,cik,irs_number,best_location_id,sic_code,industry_name,sic_sector,sector_name,source_type,source_id,num_parents,num_children,top_parent_id,company_name,max_year,min_year,no_sic
0,1,1,0,2010,20,221759452.0,1,3823.0,"Industrial instruments for measurement, displa...",3800.0,Instruments and related products,filers,579799,1,20,54894,K TRON INTERNATIONAL INC,2011,2003,0
1,2,37,0,2003,3153,630004250.0,74,4911.0,Electric services,4900.0,"Electric, gas, and sanitary services",filers,52734,6,36,37,ALABAMA POWER CO,2019,2003,0
2,3,37,0,2004,3153,630004250.0,74,4911.0,Electric services,4900.0,"Electric, gas, and sanitary services",filers,116021,6,37,37,ALABAMA POWER CO,2019,2003,0
3,4,37,0,2005,3153,630004250.0,74,4911.0,Electric services,4900.0,"Electric, gas, and sanitary services",filers,212295,6,36,37,ALABAMA POWER CO,2019,2003,0
4,5,37,0,2006,3153,630004250.0,74,4911.0,Electric services,4900.0,"Electric, gas, and sanitary services",filers,299520,6,37,37,ALABAMA POWER CO,2019,2003,0


In [8]:
company_info.shape

(4001965, 20)

In [9]:
recent_companies = company_info[company_info['most_recent'] == 1]
custom_aggregations = {'cw_id' : ['first'],
                       'company_name': ['first', 'last'],
                       'cik' : ['first'],
                       'sic_code' : ['first'],
                       'industry_name' : ['first'],
                       'sic_sector' : ['first'],
                       'sector_name' : ['first'],
                       'num_parents' : ['max'],
                       'num_children' : ['max'],
                       'top_parent_id' : ['max'],
                        'year' : ['min','count'],
                       'max_year' : ['max'],
                       'min_year' : ['min']
                      }

company_list = recent_companies.groupby('cw_id').agg(custom_aggregations)

In [10]:
company_list.columns = ["_".join(x) for x in company_list.columns.ravel()]
company_list.head()

,cw_id_first,company_name_first,company_name_last,cik_first,sic_code_first,industry_name_first,sic_sector_first,sector_name_first,num_parents_max,num_children_max,top_parent_id_max,year_min,year_count,max_year_max,min_year_min
cw_id,,,,,,,,,,,,,,,
1,1,K TRON INTERNATIONAL INC,K TRON INTERNATIONAL INC,20,3823.0,"Industrial instruments for measurement, displa...",3800.0,Instruments and related products,0,0,1,2011,1,2011,2003
2,2,AAR CORP,AAR CORP,1750,3720.0,Aircraft & parts,3700.0,Transportation equipment,0,17,2,2019,1,2019,2003
3,3,ABBOTT LABORATORIES,ABBOTT LABORATORIES,1800,2834.0,Pharmaceutical preparations,2800.0,Chemicals and allied products,0,550,3,2019,1,2019,2003
4,4,ABEL/NOSER CORP,ABEL/NOSER CORP,1841,NaN,NaN,NaN,NaN,0,0,4,2016,1,2016,2003
5,5,ABERCROMBIE & FITCH CO,ABERCROMBIE & FITCH CO,1848,5600.0,Retail-apparel & accessory stores,5600.0,Apparel and accessory stores,0,0,5,2008,1,2008,2005


## Read in relationships, create roll up of parent-child and child-parent dataset

In [12]:
relationships = pd.read_csv('corpwatch_api_tables_csv/relationships.csv', sep = "\t", escapechar='\\')

In [13]:
company_relationships = pd.read_csv('corpwatch_api_tables_csv/company_relations.csv', sep = "\t", escapechar='\\')

In [14]:
relationships_recent = relationships[relationships['year'] > 2018]

In [15]:
relationships_recent.head()

,relationship_id,company_name,location,filing_id,country_code,subdiv_code,clean_company,cik,ignore_record,parse_method,hierarchy,percent,parent_cw_id,cw_id,filer_cik,year,quarter
4094806,4094991,"PolarityTE, Inc",Nevada,16555966,US,NV,PolarityTE Inc,0.0,0,"simple table, company|location",0,NaN,16876.0,1091408.0,1076682,2019,1
4094807,4094992,"PolarityTE MD, Inc",Nevada,16555966,US,NV,PolarityTE MD Inc,0.0,0,"simple table, company|location",0,NaN,16876.0,1091409.0,1076682,2019,1
4094808,4094993,"PolarityTE RD, Inc",Nevada,16555966,US,NV,PolarityTE RD Inc,0.0,0,"simple table, company|location",0,NaN,16876.0,1091410.0,1076682,2019,1
4094809,4094994,"Utah CRO Services, Inc",Nevada,16555966,US,NV,Utah CRO Services Inc,0.0,0,"simple table, company|location",0,NaN,16876.0,1091468.0,1076682,2019,1
4094810,4094995,"IBEX Preclinical Research, Inc",Nevada,16555966,US,NV,IBEX Preclinical Research Inc,0.0,0,"simple table, company|location",0,NaN,16876.0,1091329.0,1076682,2019,1


In [19]:
parent_child_groups = relationships_recent.groupby(['parent_cw_id']).agg({
     'cw_id':lambda x: list(pd.unique(x)),
     'clean_company': lambda x: list(pd.unique(x))})


In [20]:
child_parent_groups = relationships_recent.groupby(['cw_id']).agg({
     'parent_cw_id' : lambda x: list(pd.unique(x)),
     'clean_company': lambda x: list(pd.unique(x))})

In [21]:
parent_child_groups = parent_child_groups.rename(columns= {"cw_id": "child_cw_id", 
                                     "parent_cw_id": "cw_id"})
parent_child_groups.head()

,child_cw_id,clean_company
parent_cw_id,,
2.0,"[2.0, nan, 724193.0, 724210.0, 198289.0, 19829...","[AAR CORP, AAR do Brasil Ltda, Rags of Honor L..."
3.0,"[638746.0, 122844.0, 122845.0, 122846.0, 12284...","[Abbott Biologicals LLC, Abbott Cardiovascular..."
13.0,"[124888.0, 124889.0, 124890.0, 737227.0, 83464...","[Acme United Ltd, Acme United Europe GmbH, Acm..."
16.0,"[125412.0, 125414.0, 125415.0, 125416.0, 12542...","[Service Transport Co, Ada Crude Oil Co, Ada M..."
17.0,"[752074.0, 984690.0]","[RELM Communications Inc, Tactical Capital Inv..."


In [22]:


# join the relationships with the originally company dataset
company_joined = pd.merge(company_list,
                     parent_child_groups, 
                     left_on= 'cw_id_first',
                     right_on='parent_cw_id',
            
                     how='left')

In [23]:
company_joined =  pd.merge(company_joined,
                     child_parent_groups, 
                     left_on= 'cw_id_first',
                     right_on='cw_id',
                     how='left')

In [24]:
company_joined.head()

,cw_id_first,company_name_first,company_name_last,cik_first,sic_code_first,industry_name_first,sic_sector_first,sector_name_first,num_parents_max,num_children_max,top_parent_id_max,year_min,year_count,max_year_max,min_year_min,child_cw_id,clean_company_x,parent_cw_id,clean_company_y
0,1,K TRON INTERNATIONAL INC,K TRON INTERNATIONAL INC,20,3823.0,"Industrial instruments for measurement, displa...",3800.0,Instruments and related products,0,0,1,2011,1,2011,2003,NaN,NaN,NaN,NaN
1,2,AAR CORP,AAR CORP,1750,3720.0,Aircraft & parts,3700.0,Transportation equipment,0,17,2,2019,1,2019,2003,"[2.0, nan, 724193.0, 724210.0, 198289.0, 19829...","[AAR CORP, AAR do Brasil Ltda, Rags of Honor L...",[2.0],[AAR CORP]
2,3,ABBOTT LABORATORIES,ABBOTT LABORATORIES,1800,2834.0,Pharmaceutical preparations,2800.0,Chemicals and allied products,0,550,3,2019,1,2019,2003,"[638746.0, 122844.0, 122845.0, 122846.0, 12284...","[Abbott Biologicals LLC, Abbott Cardiovascular...",NaN,NaN
3,4,ABEL/NOSER CORP,ABEL/NOSER CORP,1841,NaN,NaN,NaN,NaN,0,0,4,2016,1,2016,2003,NaN,NaN,NaN,NaN
4,5,ABERCROMBIE & FITCH CO,ABERCROMBIE & FITCH CO,1848,5600.0,Retail-apparel & accessory stores,5600.0,Apparel and accessory stores,0,0,5,2008,1,2008,2005,NaN,NaN,NaN,NaN


In [25]:
import numpy as np
company_joined.child_cw_id.replace(np.NaN, '[]', inplace=True)
company_joined.clean_company_x.replace(np.NaN, '[]', inplace=True)
company_joined['num_children_from_list'] = company_joined.apply(lambda row : len(set(row['clean_company_x'])), axis = 1)


In [26]:
company_joined.num_children_from_list

0            2
1           51
2          544
3            2
4            2
          ... 
1112825      2
1112826      2
1112827      2
1112828      2
1112829      2
Name: num_children_from_list, Length: 1112830, dtype: int64

In [ ]:
# Why 2 for the empty sets ? Seems like maybe I am doing something wrong??? .... to be continued